In [2]:
import sys, os
sys.path.append('/home/max/software/django-tmv/tmv_mcc-apsis/BasicBrowser')
sys.path.append('/home/galm/software/django/tmv/BasicBrowser/')
os.environ.setdefault("DJANGO_SETTINGS_MODULE", "BasicBrowser.settings")
#django.setup()

#from scoping.models import *
#from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from itertools import product, combinations
#from utils.text import *
#from pycountry_convert import  country_alpha2_to_continent_code, country_alpha3_to_country_alpha2

import pandas as pd
from mordecai import Geoparser
import re

In [2]:
df = pd.read_csv(r'data/finalDf.csv', encoding='UTF-8')
print(df.columns.to_list())
print(df.shape)
df.head()

['Unnamed: 0', 'id', 'content', 'title', 'relevant', 'seen', 'random', '2 - Adaptation', '2 - Mitigation', '2 - Other', '4 - 1. Nodality', '4 - 3. Treasure', '4 - 4. Authority', '4 - 4. Organisation', '7 - International', '7 - National', '7 - Subnational ', '8 - Coastal', '8 - Rivers', '8 - Terrestrial', '8 - Human', '9 - Ex-ante', '9 - Ex-post', 'y_k_prediction_SVM-rbf', '0 - relevance - prediction', 'text', 'labels', 'exclude_score', 'include_score', 'relevant_high', '2 - Adaptation_pred', '2 - Mitigation_pred', '2 - Other_pred', '4 - 1. Nodality_pred', '4 - 3. Treasure_pred', '4 - 4. Authority_pred', '4 - 4. Organisation_pred', '7 - International_pred', '7 - National_pred', '7 - Subnational _pred', '8 - Coastal_pred', '8 - Rivers_pred', '8 - Terrestrial_pred', '8 - Human_pred', '9 - Ex-ante_pred', '9 - Ex-post_pred']
(52590, 46)


,Unnamed: 0,id,content,title,relevant,seen,random,2 - Adaptation,2 - Mitigation,2 - Other,...,4 - 4. Organisation_pred,7 - International_pred,7 - National_pred,7 - Subnational _pred,8 - Coastal_pred,8 - Rivers_pred,8 - Terrestrial_pred,8 - Human_pred,9 - Ex-ante_pred,9 - Ex-post_pred
0,0,566587.0,This paper examines the use of low grade heat ...,Desalination using low grade heat in the proce...,NaN,NaN,NaN,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,467562.0,It is easy to get lost in the vast amount of k...,A mapping of global warming research based on ...,NaN,NaN,NaN,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,564250.0,The possibility of trading greenhouse gas emis...,Global change and the mulga woodlands of south...,NaN,NaN,NaN,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,790695.0,The last resort for conservation of rare tree ...,"Exploring and conserving a ""microcosm"": whole-...",NaN,NaN,NaN,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,1051089.0,With the continuous expansion of drylands in t...,Potential and environmental control of carbon ...,NaN,NaN,NaN,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
included = df[(df['relevant_high'] == 1) | (df['relevant']==1)]
len(included.id.unique())

8801

In [4]:
df = df[(df['relevant_high'] == 1) | (df['relevant']==1)]
print(df.shape)
df.head()

(8801, 46)


,Unnamed: 0,id,content,title,relevant,seen,random,2 - Adaptation,2 - Mitigation,2 - Other,...,4 - 4. Organisation_pred,7 - International_pred,7 - National_pred,7 - Subnational _pred,8 - Coastal_pred,8 - Rivers_pred,8 - Terrestrial_pred,8 - Human_pred,9 - Ex-ante_pred,9 - Ex-post_pred
10,10,505468.0,A large number of small- to medium-sized commu...,Community pond rehabilitation to deal with cli...,NaN,NaN,NaN,0.0,0.0,0.0,...,0.959119,0.008249,0.046516,0.123154,0.039962,0.874821,0.068860,0.103273,0.067134,0.057988
11,11,3286707.0,The purpose of this paper is to demonstrate th...,Vulnerability to global environmental changes:...,NaN,NaN,NaN,0.0,0.0,0.0,...,0.048519,0.007969,0.027088,0.054027,0.003420,0.005188,0.004376,0.022849,0.090995,0.109139
15,15,1962940.0,The issue of climate change has shed a new lig...,Migratory issues in climate adaptation policie...,NaN,NaN,NaN,0.0,0.0,0.0,...,0.003477,0.236410,0.458303,0.103715,0.007104,0.007903,0.014083,0.123407,0.261631,0.441075
19,19,1047270.0,(1) As on-the-ground projects come into existe...,Women's Empowerment and Climate Change Adaptat...,NaN,NaN,NaN,0.0,0.0,0.0,...,0.910039,0.020661,0.142046,0.283829,0.003045,0.003763,0.004649,0.016103,0.111538,0.292670
20,20,770190.0,Small rural coastal communities located in Atl...,Climatic and Environmental Changes Affecting C...,NaN,NaN,NaN,0.0,0.0,0.0,...,0.060801,0.016657,0.176772,0.501330,0.891267,0.044223,0.035439,0.026959,0.139272,0.180712


### Geoparse using Mordecai

In [6]:
geo = Geoparser()

geo.geoparse("I took the tube from Oxford Circus to London Bridge, via Bank")

Models path: /home/galm/software/mordecai-env/lib/python3.8/site-packages/mordecai/models/


[{'word': 'Oxford Circus',
  'spans': [{'start': 21, 'end': 34}],
  'country_predicted': 'GBR',
  'country_conf': 0.96374094,
  'geo': {'admin1': 'England',
   'lat': '51.51517',
   'lon': '-0.14181',
   'country_code3': 'GBR',
   'geonameid': '2640727',
   'place_name': 'Oxford Circus Underground Station',
   'feature_class': 'S',
   'feature_code': 'MTRO'}},
 {'word': 'London Bridge',
  'spans': [{'start': 38, 'end': 51}],
  'country_predicted': 'GBR',
  'country_conf': 0.96374094,
  'geo': {'admin1': 'England',
   'lat': '51.50821',
   'lon': '-0.08763',
   'country_code3': 'GBR',
   'geonameid': '6619889',
   'place_name': 'London Bridge',
   'feature_class': 'S',
   'feature_code': 'BDG'}}]

In [ ]:
geos = []

for i, row in df.iterrows():
    
    t = row['title'] + row['content']
    t = t.split("Copyright (C)")[0] 
    t = re.split("\([C-c]\) [1-2][0-9]{3} Elsevier",t)[0] 
    t = t.split("Published by Elsevier")[0] 
    t = t.split("Copyright. (C)")[0] 
    t = re.split("\. \(C\) [1-2][0-9]{3} ",t)[0] 
    t = re.split("\. \\xA9 [1-2][0-9]{3}", t)[0] #Copyright symbol
    t = re.split("\. \(C\) Copyright",t)[0]     
    
    gp = geo.geoparse(t)
    
    rplaces = []
    continent = None #Left out here -- will add from a table later
    for p in gp:
        try:
            a2 = country_alpha3_to_country_alpha2(p["country_predicted"])
            continent = country_alpha2_to_continent_code(a2)
        except:
            pass
        if "geo" in p:
            try:
                a2 = country_alpha3_to_country_alpha2(p["geo"]["country_code3"])
                continent = country_alpha2_to_continent_code(a2)
            except:
                pass
            p["geo"]["doc_id"] = row['id']
            p["geo"]["word"] = p["word"]
            p["geo"]["country_predicted"] = p["country_predicted"]
            p["geo"]["country_conf"] = p["country_conf"]
            geos.append(p["geo"])
        rplaces.append(p)
    df.loc[i,"continent"] = continent
    df.loc[i,"places"] = len(rplaces)
    
    
print(len(geos))

Traceback (most recent call last):
  File "/home/galm/software/mordecai-env/lib/python3.8/site-packages/mordecai/geoparse.py", line 726, in infer_country
    prediction = self.country_model.predict(i['matrix']).transpose()[0]
  File "/home/galm/software/mordecai-env/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py", line 130, in _method_wrapper
    return method(self, *args, **kwargs)
  File "/home/galm/software/mordecai-env/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py", line 1599, in predict
    tmp_batch_outputs = predict_function(iterator)
  File "/home/galm/software/mordecai-env/lib/python3.8/site-packages/tensorflow/python/eager/def_function.py", line 780, in __call__
    result = self._call(*args, **kwds)
  File "/home/galm/software/mordecai-env/lib/python3.8/site-packages/tensorflow/python/eager/def_function.py", line 814, in _call
    results = self._stateful_fn(*args, **kwds)
  File "/home/galm/software/mordecai-env/lib/python3.8

None
None


Traceback (most recent call last):
  File "/home/galm/software/mordecai-env/lib/python3.8/site-packages/mordecai/geoparse.py", line 726, in infer_country
    prediction = self.country_model.predict(i['matrix']).transpose()[0]
  File "/home/galm/software/mordecai-env/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py", line 130, in _method_wrapper
    return method(self, *args, **kwargs)
  File "/home/galm/software/mordecai-env/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py", line 1599, in predict
    tmp_batch_outputs = predict_function(iterator)
  File "/home/galm/software/mordecai-env/lib/python3.8/site-packages/tensorflow/python/eager/def_function.py", line 780, in __call__
    result = self._call(*args, **kwds)
  File "/home/galm/software/mordecai-env/lib/python3.8/site-packages/tensorflow/python/eager/def_function.py", line 814, in _call
    results = self._stateful_fn(*args, **kwds)
  File "/home/galm/software/mordecai-env/lib/python3.8

None
None
None
None


Traceback (most recent call last):
  File "/home/galm/software/mordecai-env/lib/python3.8/site-packages/mordecai/geoparse.py", line 726, in infer_country
    prediction = self.country_model.predict(i['matrix']).transpose()[0]
  File "/home/galm/software/mordecai-env/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py", line 130, in _method_wrapper
    return method(self, *args, **kwargs)
  File "/home/galm/software/mordecai-env/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py", line 1599, in predict
    tmp_batch_outputs = predict_function(iterator)
  File "/home/galm/software/mordecai-env/lib/python3.8/site-packages/tensorflow/python/eager/def_function.py", line 780, in __call__
    result = self._call(*args, **kwds)
  File "/home/galm/software/mordecai-env/lib/python3.8/site-packages/tensorflow/python/eager/def_function.py", line 814, in _call
    results = self._stateful_fn(*args, **kwds)
  File "/home/galm/software/mordecai-env/lib/python3.8

...

None
None
None


Traceback (most recent call last):
  File "/home/galm/software/mordecai-env/lib/python3.8/site-packages/mordecai/geoparse.py", line 726, in infer_country
    prediction = self.country_model.predict(i['matrix']).transpose()[0]
  File "/home/galm/software/mordecai-env/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py", line 130, in _method_wrapper
    return method(self, *args, **kwargs)
  File "/home/galm/software/mordecai-env/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py", line 1599, in predict
    tmp_batch_outputs = predict_function(iterator)
  File "/home/galm/software/mordecai-env/lib/python3.8/site-packages/tensorflow/python/eager/def_function.py", line 780, in __call__
    result = self._call(*args, **kwds)
  File "/home/galm/software/mordecai-env/lib/python3.8/site-packages/tensorflow/python/eager/def_function.py", line 814, in _call
    results = self._stateful_fn(*args, **kwds)
  File "/home/galm/software/mordecai-env/lib/python3.8

None
None


Traceback (most recent call last):
  File "/home/galm/software/mordecai-env/lib/python3.8/site-packages/mordecai/geoparse.py", line 726, in infer_country
    prediction = self.country_model.predict(i['matrix']).transpose()[0]
  File "/home/galm/software/mordecai-env/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py", line 130, in _method_wrapper
    return method(self, *args, **kwargs)
  File "/home/galm/software/mordecai-env/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py", line 1599, in predict
    tmp_batch_outputs = predict_function(iterator)
  File "/home/galm/software/mordecai-env/lib/python3.8/site-packages/tensorflow/python/eager/def_function.py", line 780, in __call__
    result = self._call(*args, **kwds)
  File "/home/galm/software/mordecai-env/lib/python3.8/site-packages/tensorflow/python/eager/def_function.py", line 814, in _call
    results = self._stateful_fn(*args, **kwds)
  File "/home/galm/software/mordecai-env/lib/python3.8

None


In [ ]:
places = pd.DataFrame.from_dict(geos)
print(places.shape)
places.head()

### Manually fixing common errors

In [ ]:
#A few are never picked up
gm_docs = df.loc[
    (df['title'].str.lower().str.contains("gulf of mexico")) |
    (df['content'].str.lower().str.contains("gulf of mexico")),
    "id"
]
geocolumns = ["word", "country_conf", "feature_code","lat","lon","place_name","feature_class","geonameid"]
gm = pd.DataFrame({"doc_id": gm_docs})
gm[geocolumns] = ["Gulf of Mexico",0.8,"GULF", 25, -90, "Gulf of Mexico", "H", 3523271]
places = pd.concat((places, gm), axis=0)


lab_docs = df.loc[
    (df['title'].str.lower().str.contains("labrador sea")) |
    (df['content'].str.lower().str.contains("labrador sea")),
    "id"
]
geocolumns = ["word", "country_conf", "feature_code","lat","lon","place_name","feature_class","geonameid"]
lab = pd.DataFrame({"doc_id": lab_docs})
lab[geocolumns] = ["Labrador Sea",0.8,"SEA", 57, -55, "Labrador Sea", "H", 3424929]
places = pd.concat((places, lab), axis=0)

baf_docs = df.loc[
    (df['title'].str.lower().str.contains("baffin bay")) |
    (df['content'].str.lower().str.contains("baffin bay")),
    "id"
]
geocolumns = ["word", "country_conf", "feature_code","lat","lon","place_name","feature_class","geonameid"]
baf = pd.DataFrame({"doc_id": baf_docs})
baf[geocolumns] = ["Baffin Bay",0.8,"BAY", 74, -68, "Baffin Bay", "H", 3831554]
places = pd.concat((places, baf), axis=0)

ok_docs = df.loc[
    (df['title'].str.lower().str.contains("sea of okhotsk")) |
    (df['content'].str.lower().str.contains("sea of okhotsk")),
    "id"
]
geocolumns = ["word", "country_conf", "feature_code","lat","lon","place_name","feature_class","geonameid"]
ok = pd.DataFrame({"doc_id": ok_docs})
ok[geocolumns] = ["Sea of Okhotsk",0.8, "SEA", 55, 150, "Sea of Okhotsk", "H", 2127380]
places = pd.concat((places, ok), axis=0)

#Mordecai only identifies this if it's spelled with the accents, but papers regularly ommit it
#Not sure if wider issue with accents. Checked São Paulo:  with and without accents are recognised but could be size bias
sp_docs = df.loc[
    (df['title'].str.lower().str.contains("sao tome and principe")) |
    (df['content'].str.lower().str.contains("sao tome and principe")),
    "id"
]
geocolumns = ["word", "country_conf", "feature_code","lat","lon","place_name","feature_class","geonameid"]
sp = pd.DataFrame({"doc_id": sp_docs})
sp[geocolumns] = ["Sao Tome",0.8, "STP", 0, 6, "Sao Tome and Principe", "PCLI", 2410758]
places = pd.concat((places, sp), axis=0)

print(places.shape)
places.tail()

In [ ]:
#A few others are mentions of placenames in relation to international agreements
#Remove these if the identified word is equal to such a placename
df['tstring'] = df['title'] + " " + df['content']
#Kyoto protocol
kyoto_docs = df.loc[
    (df['tstring'].str.lower().str.contains("kyoto target")) |
    (df['tstring'].str.lower().str.contains("kyoto process")) |
    (df['tstring'].str.lower().str.contains("kyoto emission")) |
    (df['tstring'].str.lower().str.contains("kyoto gas")) |
    (df['tstring'].str.lower().str.contains("kyoto agreement")) |
    (df['tstring'].str.lower().str.contains("kyoto protocol")) |
    (df['tstring'].str.lower().str.contains("kyoto framework")),
    "id"
]

places = places.drop(places[(places['doc_id'].isin(kyoto_docs)) & (places['word'].str.lower()=="kyoto")].index)

#Paris agreement -- remove if near COP or agreement
paris_docs = df.loc[
    (df['tstring'].str.contains('(Paris(?:\S* ){0,15}COP)|(COP(?:\S* ){0,15}Paris)')) |
    (df['tstring'].str.contains('(Paris(?:\S* ){0,15}Agreement)|(COP(?:\S* ){0,15}Agreement)')) ,
    'id'
]
places = places.drop(places[(places['doc_id'].isin(paris_docs)) & (places['word'].str.lower()=="paris")].index)

# Copenhagen
copenhagen_docs = df.loc[
    (df['tstring'].str.contains('(Copenhagen(?:\S* ){0,15}COP)|(COP(?:\S* ){0,15}Copenhagen)')) |
    (df['tstring'].str.contains('(Copenhagen(?:\S* ){0,3}Accord)|(Accord(?:\S* ){0,3}Copenhagen)')) ,
    'id'
]
places = places.drop(places[(places['doc_id'].isin(copenhagen_docs)) & (places['word'].str.lower()=="copenhagen")].index)

#Berlin
berlin_docs = df.loc[
    (df['tstring'].str.contains('(Berlin(?:\S* ){0,15}COP)|(COP(?:\S* ){0,15}Berlin)')),
    'id'
]
places = places.drop(places[(places['doc_id'].isin(berlin_docs)) & (places['word'].str.lower()=="berlin")].index)

#Cancun
cancun_docs = df.loc[
    (df['tstring'].str.contains('(Cancun(?:\S* ){0,15}COP)|(COP(?:\S* ){0,15}Cancun)')) |
    (df['tstring'].str.lower().str.contains('cancun pledge')),
    'id'
]
places = places.drop(places[(places['doc_id'].isin(cancun_docs)) & (places['word'].str.lower()=="cancun")].index)

print(places.shape)

In [ ]:
#These seem to come from errors in the Mordecai database. 
#e.g. the word "pakistan" is usually linked to the Pakistani embassy, United States is tagged as US Virgin Islands
#Just replace every occurance of these word matches

geocolumns = ["feature_code", "lat", "lon", "place_name", "feature_class", "geonameid", "country_code3"]

places.loc[places["word"]=="Pakistan", geocolumns]=["PCLI",30,70,"Islamic Republic of Pakistan","A",1168579,"PAK"]
places.loc[places["word"]=="Colombia", geocolumns]=["PCLI",4,-73.25,"Colombia","A",3686110, "COL"]
places.loc[places["word"]=="Argentina", geocolumns]=["PCLI",-34,-64,"Argentine Republic","A",3865483, "ARG"]
places.loc[places["word"]=="Sahara", geocolumns] = ["DSRT", 26, 13, "Sahara", "T", 2212709, None]
places.loc[places["word"]=="Alps",geocolumns] = ["MTS", 46.41667, 10, "Alps", "T", 2661786, None]
places.loc[places["word"]=="Mediterranean Sea",geocolumns] = ["SEA", 35, 20, "Mediterranean Sea", "T", 2661786, None]
places.loc[places["word"]=="MEDITERRANEAN",geocolumns] = ["SEA", 35, 20, "Mediterranean Sea", "T", 2661786, None]
places.loc[places["word"]=="East China",geocolumns] = ["PCLI", 35, 105, "China", "A", 1814991, "CHN"]
places.loc[places["word"]=="South China",geocolumns] = ["PCLI", 35, 105, "China", "A", 1814991, "CHN"]
places.loc[places["word"]=="Great Lakes",geocolumns] = ["LK", 45.68751, -84.43753, "Great Lakes", "H", 4994594, "USA"]
places.loc[places["word"]=="Catalonia",geocolumns] = ["ADM1", 41.82046, 1.86768, "Catalunya", "A", 3336901, "ESP"]
places.loc[places["word"]=="South Pacific",geocolumns] = ["OCN", -45, -130, "South Pacific Ocean", "H", 4030483, None]
places.loc[places["word"]=="Gulf Coast",geocolumns] = ["AREA", 29.36901, -95.00565, "Gulf Coast", "L", 7287689, "USA"]
places.loc[places["word"]=="Gulf coast",geocolumns] = ["AREA", 29.36901, -95.00565, "Gulf Coast", "L", 7287689, "USA"]
places.loc[places["word"]=="Hainan Island",geocolumns] = ["ISL", 19.2, 109.7, "Hainan Dao", "T", 1809055, "CHN"]
places.loc[places["word"]=="Red Sea",geocolumns] = ["SEA", 20.26735, 38.53455, "Red Sea", "H", 350155, None]
places.loc[places["word"]=="Himalayan",geocolumns] = ["MTS", 28,84, "Himalayas", "T", 1252558, None]
places.loc[places["word"]=="Himalayas",geocolumns] = ["MTS", 28,84, "Himalayas", "T", 1252558, None]
places.loc[places["word"]=="North America's",geocolumns] = ["CONT", 46.07323, -100.54688, "North America", "L", 6255149, None]
places.loc[places["word"]=="Atlantic Ocean",geocolumns] = ["OCN", 10, -25, "Atlantic Ocean", "H", 3373405, None]
places.loc[places["word"]=="Scandinavia",geocolumns] = ["RGN", 63, 12, "Scandinavia", "L", 2614165, None]
places.loc[places["word"]=="California (USA",geocolumns] = ["ADM1", 37.25022, -119.75126, "California", "A", 5332921, "USA"]
places.loc[places["word"]=="California, USA",geocolumns] = ["ADM1", 37.25022, -119.75126, "California", "A", 5332921, "USA"]
places.loc[places["word"]=="North Pacific",geocolumns] = ["OCN", 30, -170, "North Pacific Ocean", "H", 4030875, None]
places.loc[places["word"]=="Huai",geocolumns] = ["STM", 33.133333, 118.5, "Huai He", "H", 1807690, "CHN"]
places.loc[places["word"]=="Washington, DC",geocolumns] = ["PPLC", 38.89511, -77.03637, "Washington", "P", 4140963, "USA"]
places.loc[places["word"]=="Messinian",geocolumns] = ["ADM2", 37.25, -21.83333, "Nomos Messinias", "A", 257149, "GRC"]
places.loc[places["word"]=="Ionian Sea",geocolumns] = ["SEA", 39, 19, "Ionian Sea", "H", 2463713, None]
places.loc[places["word"]=="NYC",geocolumns] = ["PPL", 40.71427, -74.00597, "New York City", "P", 5128581, "USA"]
places.loc[places["word"]=="Indian Ocean",geocolumns] = ["OCN", -10, 70, "Indian Ocean", "P", 1545739, None]
places.loc[places["word"]=="North Sea",geocolumns] = ["SEA", 55, 3, "North Sea", "P", 2960848, None]
places.loc[places["word"]=="Philippine Sea",geocolumns] = ["SEA", 20, 135, "Philippine Sea", "P", 1818190, None]
places.loc[places["word"]=="Black Sea",geocolumns] = ["SEA", 43, 34, "Black Sea", "H", 630673, None]
places.loc[places["word"]=="Coral Sea",geocolumns] = ["SEA", -20, 155, "Coral Sea", "H", 2194166, None]
places.loc[places["word"]=="Timor Sea",geocolumns] = ["SEA", -11, 127, "Timor Sea", "H", 2078065, None]
places.loc[places["word"]=="Hudson Bay",geocolumns] = ["BAY", 60, -85, "Hudson Bay", "H", 5978134, "CAN"]
places.loc[places["word"]=="Bering Sea",geocolumns] = ["SEA", 60, -175, "Bering Sea", "H", 4031788, None]
places.loc[places["word"]=="Okhotsk Sea",geocolumns] = ["SEA", 55, 150, "Sea of Okhotsk", "H", 2127380, None]

places.loc[places["place_name"]=="Central Upper Nile",geocolumns] = ["ADM1", 10, 32.7, "Upper Nile", "A", 381229, "SSD"]
places.loc[places["place_name"]=="Gobolka Woqooyi Galbeed","place_name"] = "Woqooyi Galbeed"

places.loc[places["word"]=="Ireland", geocolumns]=["PCLI",53,-8,"Ireland","A",2963597,"IRL"]
places.loc[places["word"]=="United States", geocolumns] = ["PCLI",39.76,-98.5,"United States","A",6252001, "USA"]
places.loc[places["word"]=="Czech Republic", geocolumns] = ["PCLI",49.75,15,"Czechia","A",3077311, "CZE"]
places.loc[places["word"]=="Czechia", geocolumns] = ["PCLI",49.75,15,"Czechia","A",3077311, "CZE"]
places.loc[places["word"]=="China", geocolumns] = ["PCLI", 35, 105, "China", "A", 1814991, "CHN"]
places.loc[places["word"]=="United Arab Emirates", geocolumns] = ["PCLI", 23.75, 54.5, "United Arab Emirates", "A", 290557, "ARE"]

#places['continent'] = places['country_code3'].apply(lambda x: get_cont(x))

print(places.shape) #Shouldn't have changed

In [ ]:
#Finally, a few terms that are wrong or can't be reasonably corrected to a most likely real place, so we drop them
#In most cases, I suspect this is just because scientific documents use somewhat different terms from the "real-worl" data
places = places[places["place_name"]!="Pacific County"]
places = places.loc[places["word"]!="B.V."]
places = places[places["word"]!="Global South"]
places = places[places["word"]!="MMT"]
places = places[places["word"]!="Yellow"]
places = places[places["word"]!="Hadley"]
places = places[places["word"]!="Western North"]
places = places[places["word"]!="colonies"]
places = places[places["word"]!="TN"]
places = places[places["word"]!="NH"]
places = places[places["word"]!="Mn"]
places = places[places["word"]!="Tx"]
places = places[places["word"]!="TX"]
places = places[places["word"]!="Tn"]
places = places[places["word"]!="FL"]
places = places[places["word"]!="Spartina"]
places = places[places["word"]!="Tamarix"]
places = places[places["word"]!="Eurasia"]
places = places[places["word"]!="Phillyrea"]
places = places[places["word"]!="N-15"]
places = places[places["word"]!="LT50"]
places = places[places["word"]!="POSEIDON"]
places = places[places["word"]!="LC50"]
places = places[places["word"]!="El Nio"]
places = places[places["word"]!="La Nia"]
places = places[places["word"]!="Red"]
places = places[places["word"]!="Gulf Stream"]
places = places[places["word"].str.len()>2]
places = places[places["word"]!="NH 1"]
places = places[places["word"]!="Quercus"]


places = places[(places["word"]!="ZJP")]
places = places[(places["word"]!="MSW")]
places = places[(places["word"]!="CCS")]
places = places[(places["word"]!="Tier-3")]
places = places[(places["word"]!="N2O")]
places = places[(places["word"]!="VKT")]
places = places[(places["word"]!="OECD")]
places = places[(places["word"]!="States")]
places = places[(places["word"]!="North to South")]
places = places[(places["word"]!="Stabilising")]
places = places[(places["word"]!="Mass Railway")]
places = places[(places["word"]!="City")]

print(places.shape)

### Use a dictionary method to add in adjectives
Mordecai does not seem to pick those up. 

Depends on libraries that aren't installed in the mordecai env, so see separate file. Will just import and merge the results here

In [ ]:
adj_df = pd.read_csv('data/geos_adjectives.csv', encoding = 'utf-8')
adj_df.head()


In [ ]:
places = pd.concat((places, adj_df), axis=0)
print(places.shape)
places.tail()

### Creating an output df

In [3]:
both_All = pd.merge(df, places, how='outer', left_on='id', right_on='doc_id')
print(both_All.shape)
both_All.head()

NameError: name 'df' is not defined

In [ ]:
#Insert the geographic region (continent, annex status)
condf = pd.read_csv('data/CountryCodes.csv', encoding = 'utf-8')
print(condf.columns)

toAdd = ['Alpha-3','UN continental', 'UN statistical', 'Annex I or II']
both_All = pd.merge(both_All, condf[toAdd], how='outer', left_on = 'country_predicted', right_on = 'Alpha-3')
both_All.head()

In [17]:
#Also insert World bank region
wbdf = pd.read_csv('data/worldbank.csv', encoding='utf-8')
toAdd = ['Code','Region', 'Income group']
both_All = pd.merge(both_All, wbdf[toAdd], how='left', left_on = 'country_predicted', right_on = 'Code')
both_All.tail()

,Unnamed: 0,id,content,title,relevant,seen,random,2 - Adaptation,2 - Mitigation,2 - Other,...,word,country_predicted,country_conf,Alpha-3,UN continental,UN statistical,Annex I or II,Code,Region,Income group
12392,39442.0,4918636.0,Objective: The paper analyses the deep contrib...,What can we learn from indigenous peoples law ...,NaN,NaN,NaN,0.0,0.0,0.0,...,Sea Sámi,SHN,0.835962,SHN,Africa,Western Africa,Non-Annex I,NaN,NaN,NaN
12393,45373.0,3973815.0,The six islands and offshore Saba Bank which m...,The Dutch Antilles,NaN,NaN,NaN,0.0,0.0,0.0,...,Curacao,CUW,0.926237,CUW,Americas,Caribbean,Non-Annex I,CUW,Latin America & Caribbean,High income
12394,47723.0,695975.0,This paper uses a series of case studies to dr...,What legal recourse do non-state Islands have ...,NaN,NaN,NaN,0.0,0.0,0.0,...,Wallis and Futuna Islands,WLF,0.939117,WLF,Oceania,Polynesia,Non-Annex I,NaN,NaN,NaN
12395,51179.0,3981651.0,That climate change is a major disruptor of ru...,Disintegrating labour relations and depolitici...,NaN,NaN,NaN,0.0,0.0,0.0,...,Príncipe,STP,0.717823,STP,Africa,Middle Africa,Non-Annex I,STP,Sub-Saharan Africa,Lower middle income
12396,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,ASM,Oceania,Polynesia,Non-Annex I,NaN,NaN,NaN


In [23]:
both_All.loc[both_All['Income group'].isna(), 'Alpha-3'].value_counts()

IOT    9
ATA    6
VEN    5
REU    4
ESH    3
SJM    2
SHN    1
ASM    1
WLF    1
COK    1
Name: Alpha-3, dtype: int64

In [ ]:
#For mapping, multiple places being mentioned should all be counted seperately
#But we still want to remove mulitple mentions to exactly the same place
geo_out = both_All.drop_duplicates(subset = ['id', 'geonameid'])
print(geo_out.shape)

#Moreover, in many cases, the more general one is  a duplicate of the more specific one 
#(e.g. article mentioning both "Germany" and "Berlin" likely is studying Berlin, rather than the whole country)
#Dropping country-level mentions (feature_code="PCLI") if a more specific mention in the same country is made
geo_out = geo_out[~(  #Keep all except docs matching both...
                (both_All[['id', 'country_predicted']].duplicated(keep=False)) & #Same country predicted in one doc
                (both_All['feature_code']=="PCLI") #And the prediction is at country level
                )]

print(geo_out.shape)

In [13]:
dfi = pd.read_csv('data/CountryMappingTable.csv')
dfi.rename(columns = {'Latitude (average)': 'lat', 'Longitude (average)': 'lon'}, inplace=True)
print(dfi.columns)

Index(['Country', 'Alpha-2', 'Alpha-3', 'Numeric code', 'lat', 'lon',
       'UN continental', 'UN statistical', 'Population_2018',
       'Population_2019', 'Annex I or II', 'SIDS', 'DFID priority',
       'World Bank Income', 'World Bank Region', 'Income group'],
      dtype='object')


In [20]:
for c in geo_out['Alpha-3'].unique():
    for val in ['lat', 'lon', 'Annex I or II', 'UN statistical', 'UN continental']:
        try:
            v = dfi[dfi['Alpha-3'] == c][val].values[0]
            geo_out.loc[(geo_out['Alpha-3'] == c) & (geo_out[val].isna()), val] = v
        except:
            print(c)

nan
nan
nan
nan
nan


In [21]:
geo_out.to_csv('data/forGeoMap.csv', index=False, encoding = 'utf-8')

In [ ]:
#For STM, we want to avoid duplicates (i.e. every document should only be in the topic model once)
# -> only keep most frequent location & we only care about the continent as that's what's fed in to the STM 
from scipy.stats import mode
stmDf = both_All.groupby('id', sort=False)['UN continental'].agg(lambda x: mode(x).mode)
outdf= pd.merge(rdf, stmDf, how='left', on = 'id')
stmDf = both_All.groupby('id', sort=False)['Annex I or II'].agg(lambda x: mode(x).mode)
outdf= pd.merge(outdf, stmDf, how='left', on = 'id')

print(outdf.shape) #Should be the same as rdf
outdf.head()

In [ ]:
outdf.to_csv('data/forSTM_withRegions.csv', index=False, encoding='utf-8')

### Author locations

In [27]:
df = pd.read_csv(r'data/AuthorLocations_SeparateLines.csv', encoding='UTF-8')
print(df.columns.to_list())
print(df.shape)
df.head()

['id', 'title', 'abstract', 'wc_category', 'PY', 'authors', 'institutions', 'positions']
(31447, 8)


,id,title,abstract,wc_category,PY,authors,institutions,positions
0,1300185,Collaborative Planning in Adaptive Flood Risk ...,Flood risk management (FRM) is moving towards ...,"['Engineering, Civil; Water Resources']",2018.0,"Soderholm, K","SYKE, Finnish Environm Inst, Freshwater Ctr, P...",1
1,1300185,Collaborative Planning in Adaptive Flood Risk ...,Flood risk management (FRM) is moving towards ...,"['Engineering, Civil; Water Resources']",2018.0,"Pihlajamaki, M","SYKE, Finnish Environm Inst, Environm Policy C...",2
2,1300185,Collaborative Planning in Adaptive Flood Risk ...,Flood risk management (FRM) is moving towards ...,"['Engineering, Civil; Water Resources']",2018.0,"Pihlajamaki, M","Univ Helsinki, Dept Environm Sci, POB 65, FIN-...",2
3,1300185,Collaborative Planning in Adaptive Flood Risk ...,Flood risk management (FRM) is moving towards ...,"['Engineering, Civil; Water Resources']",2018.0,"Pihlajamaki, M","Univ Oulu, Water Resources & Environm Engn Res...",2
4,1300185,Collaborative Planning in Adaptive Flood Risk ...,Flood risk management (FRM) is moving towards ...,"['Engineering, Civil; Water Resources']",2018.0,"Dubrovin, T","SYKE, Finnish Environm Inst, Freshwater Ctr, P...",3


In [28]:
testing=False
if testing: df = df.head(50)

In [29]:
geos = []

for i, row in df.iterrows():
    
    t = row['institutions']    
    
    gp = geo.geoparse(t)
    
    rplaces = []
    continent = None 
    for p in gp:
        try:
            a2 = country_alpha3_to_country_alpha2(p["country_predicted"])
            continent = country_alpha2_to_continent_code(a2)
        except:
            pass
        if "geo" in p:
            try:
                a2 = country_alpha3_to_country_alpha2(p["geo"]["country_code3"])
                continent = country_alpha2_to_continent_code(a2)
            except:
                pass
            p["geo"]["doc_id"] = row['id']
            p["geo"]["author"] = row['authors']
            p["geo"]["position"] = row['positions']
            p["geo"]["institution"] = row['institutions']
            p["geo"]["word"] = p["word"]
            p["geo"]["country_predicted"] = p["country_predicted"]
            p["geo"]["country_conf"] = p["country_conf"]
            geos.append(p["geo"])
        rplaces.append(p)
    df.loc[i,"continent"] = continent
    df.loc[i,"places"] = len(rplaces)
    
    
print(len(geos))

Traceback (most recent call last):
  File "/home/galm/software/mordecai-env/lib/python3.8/site-packages/mordecai/geoparse.py", line 726, in infer_country
    prediction = self.country_model.predict(i['matrix']).transpose()[0]
  File "/home/galm/software/mordecai-env/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py", line 130, in _method_wrapper
    return method(self, *args, **kwargs)
  File "/home/galm/software/mordecai-env/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py", line 1599, in predict
    tmp_batch_outputs = predict_function(iterator)
  File "/home/galm/software/mordecai-env/lib/python3.8/site-packages/tensorflow/python/eager/def_function.py", line 780, in __call__
    result = self._call(*args, **kwds)
  File "/home/galm/software/mordecai-env/lib/python3.8/site-packages/tensorflow/python/eager/def_function.py", line 814, in _call
    results = self._stateful_fn(*args, **kwds)
  File "/home/galm/software/mordecai-env/lib/python3.8

None
None
None


Traceback (most recent call last):
  File "/home/galm/software/mordecai-env/lib/python3.8/site-packages/mordecai/geoparse.py", line 726, in infer_country
    prediction = self.country_model.predict(i['matrix']).transpose()[0]
  File "/home/galm/software/mordecai-env/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py", line 130, in _method_wrapper
    return method(self, *args, **kwargs)
  File "/home/galm/software/mordecai-env/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py", line 1599, in predict
    tmp_batch_outputs = predict_function(iterator)
  File "/home/galm/software/mordecai-env/lib/python3.8/site-packages/tensorflow/python/eager/def_function.py", line 780, in __call__
    result = self._call(*args, **kwds)
  File "/home/galm/software/mordecai-env/lib/python3.8/site-packages/tensorflow/python/eager/def_function.py", line 814, in _call
    results = self._stateful_fn(*args, **kwds)
  File "/home/galm/software/mordecai-env/lib/python3.8

None
None
None
None


Traceback (most recent call last):
  File "/home/galm/software/mordecai-env/lib/python3.8/site-packages/mordecai/geoparse.py", line 726, in infer_country
    prediction = self.country_model.predict(i['matrix']).transpose()[0]
  File "/home/galm/software/mordecai-env/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py", line 130, in _method_wrapper
    return method(self, *args, **kwargs)
  File "/home/galm/software/mordecai-env/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py", line 1599, in predict
    tmp_batch_outputs = predict_function(iterator)
  File "/home/galm/software/mordecai-env/lib/python3.8/site-packages/tensorflow/python/eager/def_function.py", line 780, in __call__
    result = self._call(*args, **kwds)
  File "/home/galm/software/mordecai-env/lib/python3.8/site-packages/tensorflow/python/eager/def_function.py", line 814, in _call
    results = self._stateful_fn(*args, **kwds)
  File "/home/galm/software/mordecai-env/lib/python3.8

...

None
None
None


Traceback (most recent call last):
  File "/home/galm/software/mordecai-env/lib/python3.8/site-packages/mordecai/geoparse.py", line 726, in infer_country
    prediction = self.country_model.predict(i['matrix']).transpose()[0]
  File "/home/galm/software/mordecai-env/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py", line 130, in _method_wrapper
    return method(self, *args, **kwargs)
  File "/home/galm/software/mordecai-env/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py", line 1599, in predict
    tmp_batch_outputs = predict_function(iterator)
  File "/home/galm/software/mordecai-env/lib/python3.8/site-packages/tensorflow/python/eager/def_function.py", line 780, in __call__
    result = self._call(*args, **kwds)
  File "/home/galm/software/mordecai-env/lib/python3.8/site-packages/tensorflow/python/eager/def_function.py", line 814, in _call
    results = self._stateful_fn(*args, **kwds)
  File "/home/galm/software/mordecai-env/lib/python3.8

None
None
None


Traceback (most recent call last):
  File "/home/galm/software/mordecai-env/lib/python3.8/site-packages/mordecai/geoparse.py", line 726, in infer_country
    prediction = self.country_model.predict(i['matrix']).transpose()[0]
  File "/home/galm/software/mordecai-env/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py", line 130, in _method_wrapper
    return method(self, *args, **kwargs)
  File "/home/galm/software/mordecai-env/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py", line 1599, in predict
    tmp_batch_outputs = predict_function(iterator)
  File "/home/galm/software/mordecai-env/lib/python3.8/site-packages/tensorflow/python/eager/def_function.py", line 780, in __call__
    result = self._call(*args, **kwds)
  File "/home/galm/software/mordecai-env/lib/python3.8/site-packages/tensorflow/python/eager/def_function.py", line 814, in _call
    results = self._stateful_fn(*args, **kwds)
  File "/home/galm/software/mordecai-env/lib/python3.8

None
None
None
53072


In [30]:
places = pd.DataFrame.from_dict(geos)
print(places.shape)
places.head()

(53072, 15)


,admin1,lat,lon,country_code3,geonameid,place_name,feature_class,feature_code,doc_id,author,position,institution,word,country_predicted,country_conf
0,Uusimaa,60.17556,24.93417,FIN,658226,Helsinki,A,ADM3,1300185,"Soderholm, K",1,"SYKE, Finnish Environm Inst, Freshwater Ctr, P...",Helsinki,FIN,0.948191
1,NA,64,26,FIN,660013,Republic of Finland,A,PCLI,1300185,"Soderholm, K",1,"SYKE, Finnish Environm Inst, Freshwater Ctr, P...",Finland,FIN,0.948191
2,Uusimaa,60.17556,24.93417,FIN,658226,Helsinki,A,ADM3,1300185,"Pihlajamaki, M",2,"SYKE, Finnish Environm Inst, Environm Policy C...",Helsinki,FIN,0.948191
3,NA,64,26,FIN,660013,Republic of Finland,A,PCLI,1300185,"Pihlajamaki, M",2,"SYKE, Finnish Environm Inst, Environm Policy C...",Finland,FIN,0.948191
4,NA,64,26,FIN,660013,Republic of Finland,A,PCLI,1300185,"Pihlajamaki, M",2,"Univ Helsinki, Dept Environm Sci, POB 65, FIN-...",Finland,FIN,0.951695


In [31]:
#Most of the manual fixes aren't relevant for author addresses
sp_docs = places.loc[places['institution'].str.lower().str.contains("sao tome and principe"),"doc_id"]

geocolumns = ["word", "country_conf", "feature_code","lat","lon","place_name","feature_class","geonameid"]
sp = pd.DataFrame({"doc_id": sp_docs})
sp[geocolumns] = ["Sao Tome",0.8, "STP", 0, 6, "Sao Tome and Principe", "PCLI", 2410758]
places = pd.concat((places, sp), axis=0)

print(places.shape)

geocolumns = ["feature_code", "lat", "lon", "place_name", "feature_class", "geonameid", "country_code3"]

places.loc[places["word"]=="Pakistan", geocolumns]=["PCLI",30,70,"Islamic Republic of Pakistan","A",1168579,"PAK"]
places.loc[places["word"]=="Colombia", geocolumns]=["PCLI",4,-73.25,"Colombia","A",3686110, "COL"]
places.loc[places["word"]=="Argentina", geocolumns]=["PCLI",-34,-64,"Argentine Republic","A",3865483, "ARG"]
places.loc[places["word"]=="Sahara", geocolumns] = ["DSRT", 26, 13, "Sahara", "T", 2212709, None]
places.loc[places["word"]=="Alps",geocolumns] = ["MTS", 46.41667, 10, "Alps", "T", 2661786, None]
places.loc[places["word"]=="Mediterranean Sea",geocolumns] = ["SEA", 35, 20, "Mediterranean Sea", "T", 2661786, None]
places.loc[places["word"]=="MEDITERRANEAN",geocolumns] = ["SEA", 35, 20, "Mediterranean Sea", "T", 2661786, None]
places.loc[places["word"]=="East China",geocolumns] = ["PCLI", 35, 105, "China", "A", 1814991, "CHN"]
places.loc[places["word"]=="South China",geocolumns] = ["PCLI", 35, 105, "China", "A", 1814991, "CHN"]
places.loc[places["word"]=="Great Lakes",geocolumns] = ["LK", 45.68751, -84.43753, "Great Lakes", "H", 4994594, "USA"]
places.loc[places["word"]=="Catalonia",geocolumns] = ["ADM1", 41.82046, 1.86768, "Catalunya", "A", 3336901, "ESP"]
places.loc[places["word"]=="South Pacific",geocolumns] = ["OCN", -45, -130, "South Pacific Ocean", "H", 4030483, None]
places.loc[places["word"]=="Gulf Coast",geocolumns] = ["AREA", 29.36901, -95.00565, "Gulf Coast", "L", 7287689, "USA"]
places.loc[places["word"]=="Gulf coast",geocolumns] = ["AREA", 29.36901, -95.00565, "Gulf Coast", "L", 7287689, "USA"]
places.loc[places["word"]=="Hainan Island",geocolumns] = ["ISL", 19.2, 109.7, "Hainan Dao", "T", 1809055, "CHN"]
places.loc[places["word"]=="Red Sea",geocolumns] = ["SEA", 20.26735, 38.53455, "Red Sea", "H", 350155, None]
places.loc[places["word"]=="Himalayan",geocolumns] = ["MTS", 28,84, "Himalayas", "T", 1252558, None]
places.loc[places["word"]=="Himalayas",geocolumns] = ["MTS", 28,84, "Himalayas", "T", 1252558, None]
places.loc[places["word"]=="North America's",geocolumns] = ["CONT", 46.07323, -100.54688, "North America", "L", 6255149, None]
places.loc[places["word"]=="Atlantic Ocean",geocolumns] = ["OCN", 10, -25, "Atlantic Ocean", "H", 3373405, None]
places.loc[places["word"]=="Scandinavia",geocolumns] = ["RGN", 63, 12, "Scandinavia", "L", 2614165, None]
places.loc[places["word"]=="California (USA",geocolumns] = ["ADM1", 37.25022, -119.75126, "California", "A", 5332921, "USA"]
places.loc[places["word"]=="California, USA",geocolumns] = ["ADM1", 37.25022, -119.75126, "California", "A", 5332921, "USA"]
places.loc[places["word"]=="North Pacific",geocolumns] = ["OCN", 30, -170, "North Pacific Ocean", "H", 4030875, None]
places.loc[places["word"]=="Huai",geocolumns] = ["STM", 33.133333, 118.5, "Huai He", "H", 1807690, "CHN"]
places.loc[places["word"]=="Washington, DC",geocolumns] = ["PPLC", 38.89511, -77.03637, "Washington", "P", 4140963, "USA"]
places.loc[places["word"]=="Messinian",geocolumns] = ["ADM2", 37.25, -21.83333, "Nomos Messinias", "A", 257149, "GRC"]
places.loc[places["word"]=="Ionian Sea",geocolumns] = ["SEA", 39, 19, "Ionian Sea", "H", 2463713, None]
places.loc[places["word"]=="NYC",geocolumns] = ["PPL", 40.71427, -74.00597, "New York City", "P", 5128581, "USA"]
places.loc[places["word"]=="Indian Ocean",geocolumns] = ["OCN", -10, 70, "Indian Ocean", "P", 1545739, None]
places.loc[places["word"]=="North Sea",geocolumns] = ["SEA", 55, 3, "North Sea", "P", 2960848, None]
places.loc[places["word"]=="Philippine Sea",geocolumns] = ["SEA", 20, 135, "Philippine Sea", "P", 1818190, None]
places.loc[places["word"]=="Black Sea",geocolumns] = ["SEA", 43, 34, "Black Sea", "H", 630673, None]
places.loc[places["word"]=="Coral Sea",geocolumns] = ["SEA", -20, 155, "Coral Sea", "H", 2194166, None]
places.loc[places["word"]=="Timor Sea",geocolumns] = ["SEA", -11, 127, "Timor Sea", "H", 2078065, None]
places.loc[places["word"]=="Hudson Bay",geocolumns] = ["BAY", 60, -85, "Hudson Bay", "H", 5978134, "CAN"]
places.loc[places["word"]=="Bering Sea",geocolumns] = ["SEA", 60, -175, "Bering Sea", "H", 4031788, None]
places.loc[places["word"]=="Okhotsk Sea",geocolumns] = ["SEA", 55, 150, "Sea of Okhotsk", "H", 2127380, None]

places.loc[places["place_name"]=="Central Upper Nile",geocolumns] = ["ADM1", 10, 32.7, "Upper Nile", "A", 381229, "SSD"]
places.loc[places["place_name"]=="Gobolka Woqooyi Galbeed","place_name"] = "Woqooyi Galbeed"

places.loc[places["word"]=="Ireland", geocolumns]=["PCLI",53,-8,"Ireland","A",2963597,"IRL"]
places.loc[places["word"]=="United States", geocolumns] = ["PCLI",39.76,-98.5,"United States","A",6252001, "USA"]
places.loc[places["word"]=="Czech Republic", geocolumns] = ["PCLI",49.75,15,"Czechia","A",3077311, "CZE"]
places.loc[places["word"]=="Czechia", geocolumns] = ["PCLI",49.75,15,"Czechia","A",3077311, "CZE"]
places.loc[places["word"]=="China", geocolumns] = ["PCLI", 35, 105, "China", "A", 1814991, "CHN"]
places.loc[places["word"]=="United Arab Emirates", geocolumns] = ["PCLI", 23.75, 54.5, "United Arab Emirates", "A", 290557, "ARE"]

#places['continent'] = places['country_code3'].apply(lambda x: get_cont(x))

print(places.shape) #Shouldn't have changed

places = places[places["place_name"]!="Pacific County"]
places = places.loc[places["word"]!="B.V."]
places = places[places["word"]!="Global South"]
places = places[places["word"]!="MMT"]
places = places[places["word"]!="Yellow"]
places = places[places["word"]!="Hadley"]
places = places[places["word"]!="Western North"]
places = places[places["word"]!="colonies"]
places = places[places["word"]!="TN"]
places = places[places["word"]!="NH"]
places = places[places["word"]!="Mn"]
places = places[places["word"]!="Tx"]
places = places[places["word"]!="TX"]
places = places[places["word"]!="Tn"]
places = places[places["word"]!="FL"]
places = places[places["word"]!="Spartina"]
places = places[places["word"]!="Tamarix"]
places = places[places["word"]!="Eurasia"]
places = places[places["word"]!="Phillyrea"]
places = places[places["word"]!="N-15"]
places = places[places["word"]!="LT50"]
places = places[places["word"]!="POSEIDON"]
places = places[places["word"]!="LC50"]
places = places[places["word"]!="El Nio"]
places = places[places["word"]!="La Nia"]
places = places[places["word"]!="Red"]
places = places[places["word"]!="Gulf Stream"]
places = places[places["word"].str.len()>2]
places = places[places["word"]!="NH 1"]
places = places[places["word"]!="Quercus"]


places = places[(places["word"]!="ZJP")]
places = places[(places["word"]!="MSW")]
places = places[(places["word"]!="CCS")]
places = places[(places["word"]!="Tier-3")]
places = places[(places["word"]!="N2O")]
places = places[(places["word"]!="VKT")]
places = places[(places["word"]!="OECD")]
places = places[(places["word"]!="States")]
places = places[(places["word"]!="North to South")]
places = places[(places["word"]!="Stabilising")]
places = places[(places["word"]!="Mass Railway")]
places = places[(places["word"]!="City")]

print(places.shape) #Likely smaller

(53072, 15)
(53072, 15)
(52489, 15)


In [32]:
#Insert the geographic region (continent, annex status)
condf = pd.read_csv('data/CountryCodes.csv', encoding = 'utf-8')
print(condf.columns)

toAdd = ['Alpha-3','UN continental', 'UN statistical', 'Annex I or II']
both_All = pd.merge(places, condf[toAdd], how='outer', left_on = 'country_predicted', right_on = 'Alpha-3')
both_All.dropna(subset=['doc_id'], inplace=True) #Remove countries in the table that don't match any document
print(both_All.shape)

#Also insert World bank region
wbdf = pd.read_csv('data/worldbank.csv', encoding='utf-8')
toAdd = ['Code','Region', 'Income group']
both_All = pd.merge(both_All, wbdf[toAdd], how='left', left_on = 'country_predicted', right_on = 'Code')
both_All.dropna(subset=['doc_id'], inplace=True)
print(both_All.shape)
both_All.tail()

Index(['Country', 'Alpha-2', 'Alpha-3', 'Numeric code', 'Latitude (average)',
       'Longitude (average)', 'UN continental', 'UN statistical',
       'Population_2018', 'Population_2019', 'Annex I or II'],
      dtype='object')
(52489, 19)
(52489, 22)


,admin1,lat,lon,country_code3,geonameid,place_name,feature_class,feature_code,doc_id,author,...,word,country_predicted,country_conf,Alpha-3,UN continental,UN statistical,Annex I or II,Code,Region,Income group
52484,NA,-29.5,28.25,LSO,932692,Kingdom of Lesotho,A,PCLI,4916290.0,"Likoetla, P",...,Lesotho,LSO,0.948191,LSO,Africa,Southern Africa,Non-Annex I,LSO,Sub-Saharan Africa,Lower middle income
52485,NA,11.82917,42.27111,DJI,224672,Bakar re Alayla,H,STM,3820251.0,"Diop, S",...,Dakar Rp,DJI,0.852964,DJI,Africa,Eastern Africa,Non-Annex I,DJI,Middle East & North Africa,Lower middle income
52486,Saint George,15.30174,-61.38808,DMA,3575635,Roseau,P,PPLC,2521391.0,"St Ville, S",...,Roseau,DMA,0.775636,DMA,Americas,Caribbean,Non-Annex I,DMA,Latin America & Caribbean,Upper middle income
52487,NA,4.85871,31.59162,SSD,11550569,Jubek,A,ADM1,696598.0,"Eltoum, SO",...,Juba,SSD,0.813797,SSD,Africa,Eastern Africa,Non-Annex I,SSD,Sub-Saharan Africa,Low income
52488,NA,-70.55,-15,ATA,4036755,Explora Escarpment,U,ESCU,1672333.0,"Fink, JH",...,Explorat,ATA,0.952811,ATA,Antartica,Antartica,Non-Annex I,NaN,NaN,NaN


In [38]:
both_All.to_csv('data/AuthorLocations_output.csv', encoding='utf-8', index=False)

In [44]:
#For mapping, multiple places being mentioned should all be counted seperately
#But we still want to remove mulitple mentions to exactly the same place
geo_out = both_All.drop_duplicates(subset = ['doc_id', 'geonameid', 'position'])
print(geo_out.shape)

#Here too, we mostly care about the most specific mention possible ()
#Dropping country-level mentions (feature_code="PCLI") if a more specific mention in the same country is made
geo_out = geo_out[~(  #Keep all except docs matching both...
                (both_All[['doc_id', 'country_predicted', 'position']].duplicated(keep=False)) & #Same country predicted in one doc
                (both_All['feature_code']=="PCLI") #And the prediction is at country level
                )].sort_values(by=['doc_id', 'position'])

print(geo_out.shape)
geo_out.to_csv('data/AuthorLocations_output_Filtered.csv', encoding='utf-8', index=False)

(46345, 22)
(32250, 22)


<ipython-input-44-9fd56cf7d102>:8: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  geo_out = geo_out[~(  #Keep all except docs matching both...
